# Öffentlich-rechtlicher Rundfunk

In [1]:
# loading libs
import pandas as pd
import numpy as np
import pprint as pp
import datetime
import Stemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import Stemmer

In [2]:
# politiker = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/politikR_demojized_reinigung.csv",
#                         index_col=0)

öffentlicher = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/örr_demojized.csv",
                           index_col=0)
öffentlicher = öffentlicher.drop(columns='...1')

# überregionale = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/überregio_demojized.csv",
#                             index_col=0)
# überregionale = überregionale.drop(columns='...1')

# digitale = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/digitale_demojized.csv",
#                        index_col=0)
# digitale = digitale.drop(columns='...1')

In [3]:
categoriesÖffentlicher = {
    'multimedia' : ['live', 'medien', 'internet', 'livestream', 'radio', 'video', 'netz', 'sozialen', 'facebook', 'fernsehen',
                    'ardmediathek', 'doku', 'mediathek', 'online', 'film', 'musik'],
    
    'polizeinachrichten' : ['polizei', 'ermittelt', 'festgenommen', 'polizisten', 'täter', 'ermittlungen', 'fall', 'zeugen',
                            'getötet', 'mutmaßlichen', 'tödlichen', 'mutmaßliche', 'ermittler', 'verdacht', 'kusel'],
    
    'wetterKlima' : ['wasser', 'sommer', 'grad', 'hitze', 'wetter', 'trockenheit', 'klimawandel', 'temperaturen', 'regen',
                     'luft', 'sonne', 'sturm', 'unwetter', 'schäden', 'gewitter', 'bäume', 'starkregen', 'zeynep'],
    
    'brandenburgBerlin' : ['berlin', 'brandenburg', 'berliner', 'potsdam', 'brandenburger', 'giffey', 'berlins', 'rbb24',
                           'hauptstadt', 'neukölln', 'cottbus'],
    
    'energiekrise' : ['gas', 'habeck', 'wirtschaftsminister', 'öl', 'bundeswirtschaftsminister', 'energiewende', 'erdgas',
                      '2030', 'gaslieferungen', 'schwedt', 'betreiber', 'preise', 'energiekrise', 'energie', 'energiepreise',
                      'strom', 'inflation', 'teurer', 'kosten', 'sparen', 'steigenden', 'steigende', 'kunden', 'energiekosten',
                      'gas', 'verbraucher'],
    
    'unglückBrände' : ['feuerwehr', 'brand', 'feuer', 'waldbrand', 'wald', 'flammen', 'ausgebrochen', 'gelöscht'],
    
    # 'coronaProtest' : ['demonstriert', 'corona-maßnahmen', 'protest'],
    
    'ukraine' : ['russland', 'ukraine', 'russischen', 'putin', 'russische', 'ukrainische', 'ukrainischen', 'kiew',
                 'russlands', 'selenskyj', 'sanktionen', 'ukraine', 'krieg', 'geflüchtete', 'geflüchteten', 'flüchtlinge',
                 'ukrainekrieg', 'flucht', 'krieges', 'ukrainer'],
    
    'covid' : ['inzidenz', 'neuinfektionen', 'corona', 'fälle', 'rki', 'corona-fälle', 'maskenpflicht', 'corona', 'corona-regeln',
               'lockerungen', 'corona-maßnahmen', 'maske', 'corona', 'coronavirus', 'lauterbach', 'impfung', 'omikron',
               'getestet', 'positiv', 'bundesgesundheitsminister', 'quarantäne', 'gesundheitsminister', 'impfen', 'corona-ticker',
               'omikron-variante', 'geimpft', 'impfstoff', 'isolation', 'corona-impfung'],
    
    'finanziellesInvest' : ['euro', 'millionen', 'geld', 'milliarden', 'kosten', 'mio', 'dollar'],
    
    'bayern' : ['bayern', 'smallbluediamond', 'bayerns', 'bayerischen', 'söder', 'bayerische', 'csu', 'br24live', 'augsburg'],
    
    'politikNational' : ['bundesregierung', 'bund', 'länder', 'bundestag', 'ländern', 'lindner', 'entlastungspaket',
                         'landesregierung', 'gesundheitsminister', 'finanzminister', 'spd', 'cdu', 'ministerpräsident',
                         'grünen', 'grüne', 'landtag', 'fdp', 'partei', 'afd', 'landtagswahl', 'innenminister', 'linke',
                         'bürgermeister', 'oberbürgermeister', 'schleswig-holstein', 'parteien', 'merz'],
    
    'unglückUnfall' : ['verletzt', 'unfall', 'krankenhaus', 'verletzte', 'gefahren'],
    
    'streiks' : ['gewerkschaft', 'personal', 'beschäftigte', 'beschäftigten', 'verdi', 'lufthansa', 'streik',
                 'warnstreik', 'warnstreiks'],
    
    'katastrophen' : ['flutkatastrophe', 'ahrtal', 'fischsterben', 'untersuchungsausschuss'],
    
    'öpnvVerkehr' : ['bahn', '9euroticket', 'züge', 'öpnv', 'bus', 'zug', 'nahverkehr', 'bahnhof', '9-euro-ticket'],
    
    'politikInternational' : ['scholz', 'bundeskanzler', 'usa', 'waffen', 'kanzler', 'regierung', 'europa', 'bundeswehr',
                              'baerbock', 'waffenlieferungen', 'außenministerin'],
    
    'justiz' : ['landgericht', 'prozess', 'verurteilt', 'gericht', 'haft', 'urteil', 'fall', 'staatsanwaltschaft', 'verfahren',
                'vorgeworfen', 'anklage', 'klage']
}

In [4]:
pp.pprint(categoriesÖffentlicher)

{'bayern': ['bayern',
            'smallbluediamond',
            'bayerns',
            'bayerischen',
            'söder',
            'bayerische',
            'csu',
            'br24live',
            'augsburg'],
 'brandenburgBerlin': ['berlin',
                       'brandenburg',
                       'berliner',
                       'potsdam',
                       'brandenburger',
                       'giffey',
                       'berlins',
                       'rbb24',
                       'hauptstadt',
                       'neukölln',
                       'cottbus'],
 'covid': ['inzidenz',
           'neuinfektionen',
           'corona',
           'fälle',
           'rki',
           'corona-fälle',
           'maskenpflicht',
           'corona',
           'corona-regeln',
           'lockerungen',
           'corona-maßnahmen',
           'maske',
           'corona',
           'coronavirus',
           'lauterbach',
           'impfung',
         

In [9]:
dfsÖffentlicher = []
# stemmer=Stemmer.Stemmer('de')

for key,values in categoriesÖffentlicher.items():
    words=pd.DataFrame({'category': key, 'term': values}) ## stemmer löschen, wenn ganze wörter gewollt sind
    dfsÖffentlicher.append(words)

termsÖffentlicher = pd.concat(dfsÖffentlicher)
termsÖffentlicher

,category,term
0,multimedia,live
1,multimedia,medien
2,multimedia,internet
3,multimedia,livestream
4,multimedia,radio
...,...,...
7,justiz,staatsanwaltschaft
8,justiz,verfahren
9,justiz,vorgeworfen
10,justiz,anklage


In [10]:
# Take the 'term' column from our list of terms
termlistÖffentlicher = list(set(termsÖffentlicher.term))

# binary=True only does 0/1
# vocabulary= is the list of words we're interested in tracking
vectorizer = TfidfVectorizer(binary=False, vocabulary = termlistÖffentlicher)

matrix = vectorizer.fit_transform(öffentlicher.text.astype('U'), )

words_df = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names_out())

In [11]:
# Group the terms by category, then loop through each category
for category_name, rows in termsÖffentlicher.groupby('category'):
    # Convert the terms for that category into a simple list
    # for example, ['student', 'educ', 'teacher']
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    öffentlicher[category_name] = words_df[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
    #öffentlicher[category_name] = öffentlicher[category_name].astype('Int64', errors='ignore')

Topic: bayern 
Worte: ['bayern', 'smallbluediamond', 'bayerns', 'bayerischen', 'söder', 'bayerische', 'csu', 'br24live', 'augsburg'] 

Topic: brandenburgBerlin 
Worte: ['berlin', 'brandenburg', 'berliner', 'potsdam', 'brandenburger', 'giffey', 'berlins', 'rbb24', 'hauptstadt', 'neukölln', 'cottbus'] 

Topic: covid 
Worte: ['inzidenz', 'neuinfektionen', 'corona', 'fälle', 'rki', 'corona-fälle', 'maskenpflicht', 'corona', 'corona-regeln', 'lockerungen', 'corona-maßnahmen', 'maske', 'corona', 'coronavirus', 'lauterbach', 'impfung', 'omikron', 'getestet', 'positiv', 'bundesgesundheitsminister', 'quarantäne', 'gesundheitsminister', 'impfen', 'corona-ticker', 'omikron-variante', 'geimpft', 'impfstoff', 'isolation', 'corona-impfung'] 

Topic: energiekrise 
Worte: ['gas', 'habeck', 'wirtschaftsminister', 'öl', 'bundeswirtschaftsminister', 'energiewende', 'erdgas', '2030', 'gaslieferungen', 'schwedt', 'betreiber', 'preise', 'energiekrise', 'energie', 'energiepreise', 'strom', 'inflation', 'teur

In [13]:
öffentlicher.groupby('user').ukraine.sum().sort_values(ascending=False)

user
BR24               3428
MDRAktuell         3105
WDRaktuell         1221
rbb24               869
rbb24Inforadio      665
DeutscheWelle       420
NDRinfo             352
SWRAktuellBW        321
DLF                 255
NDRnds              224
butenunbinnen       223
hessenschau         199
ndr                 189
SWRAktuellRP        181
mdr_th              165
MDR_SN              156
SRaktuell           114
MDR_SAN             107
WDR                 101
rbbabendschau        98
NDRsh                88
MDRpresse            41
SWRpresse            34
ARTEde               27
dlfkultur            24
hrPresse             14
BR_Presse            13
SRKommunikation       7
mdrde                 7
dlfnova               3
Name: ukraine, dtype: int64

In [14]:
öffentlicher.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,multimedia,politikInternational,politikNational,polizeinachrichten,streiks,ukraine,unglückBrände,unglückUnfall,wetterKlima,öpnvVerkehr
user,,,,,,,,,,,,,,,,,,,,,
ARTEde,2022-10-31T13:28:06Z2022-10-31T09:46:48Z2022-1...,9.112967e+20,Im Zeitalter der #Selfies erscheinen uns die S...,2169,6503,73344,1051,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/jagodamarinic/status/15844...,...,180,9,9,6,1,27,2,3,7,3
BR24,2022-10-31T21:08:00Z2022-10-31T20:30:00Z2022-1...,2.711755e+22,Auf dem Rückweg von einem #Freizeitpark am Woc...,131626,80746,270054,25872,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,https://twitter.com/jannikpentz/status/1583756...,...,779,1230,2160,1207,212,3428,338,522,841,468
BR_Presse,2022-10-28T16:24:18Z2022-10-28T12:11:03Z2022-1...,3.561641e+20,Diesen Mann muss man lieben: Mit sanfter Stimm...,449,549,1880,114,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/BR24/status/15849709755982...,...,75,4,2,3,1,13,1,1,3,0
DLF,2022-10-31T07:57:53Z2022-10-31T07:40:48Z2022-1...,2.512554e+21,@guillaumemarin7 @spdde Höre dir gerne das kom...,4276,3984,16785,1238,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/CIVISPreis/status/15326812...,...,38,87,216,28,3,255,4,3,18,5
DeutscheWelle,2022-10-31T15:31:00Z2022-10-31T14:02:00Z2022-1...,2.735969e+21,"Die Infrastruktur in Cottbus ist überlastet: ""...",3154,5287,12744,658,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/dwnews/status/158571344150...,...,114,123,78,24,4,420,6,5,23,9
MDRAktuell,2022-10-31T21:41:49Z2022-10-31T20:50:00Z2022-1...,2.104257e+22,Die Internationale Atomenergiebehörde untersuc...,84167,87290,347702,24436,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/AufstandLastGen/status/158...,...,475,1336,2384,1055,290,3105,336,397,398,318
MDR_SAN,2022-10-31T19:58:36Z2022-10-31T13:27:24Z2022-1...,2.783265e+21,Am Reformationstag hatten in Wittenberg unter ...,2088,3608,10390,798,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,https://twitter.com/sebwilken/status/158523592...,...,31,14,186,135,28,107,72,65,69,45
MDR_SN,2022-10-31T14:27:12Z2022-10-31T13:55:39Z2022-1...,4.099693e+21,Die Hauptsaison der #Dresdner #Weißen #Flotte ...,3915,5713,13988,1261,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/SportimOsten/status/156389...,...,47,31,221,285,49,156,152,82,142,51
MDRpresse,2022-10-30T08:50:03Z2022-10-28T09:28:19Z2022-1...,7.660581e+20,Zu #Halloween in den #Harz: Der Magdeburger @P...,448,1073,3021,210,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/HarzKurier/status/15851708...,...,235,8,3,8,2,41,8,1,5,2


In [15]:
öffentlicherCategorized = öffentlicher
öffentlicherCategorized.to_csv("../../methods/representative-speech/mediendaten/zuordnungenBundesländer/öffentlicherCategorized.csv")